In [3]:
!ls

drive  sample_data


In [7]:
%cd /content/drive/MyDrive/NLP


/content/drive/MyDrive/NLP


In [11]:
! git clone  https://github.com/avinashnair02/Hotel_Review_Classifier_

Cloning into 'Hotel_Review_Classifier_'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 45 (delta 11), reused 45 (delta 11), pack-reused 0
Unpacking objects: 100% (45/45), done.
Checking out files: 100% (45/45), done.


In [83]:
import sys
sys.path.append('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/Helpers_NN.py')

In [84]:
import Helpers_NN


In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.preprocessing import sequence
from tensorflow.python.keras import Sequential
from keras.models import load_model
from tensorflow.python.keras.layers import Dense, Embedding, GlobalAveragePooling1D

import pickle


from Helpers_NN import add_sum_suffix, text_cleanup, reverse_encode, add_one_argmax_score, conf_matrix

In [86]:
sns.set_context('poster')

In [87]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/structured.csv')
df.review_summary=df.astype(str)

In [89]:
df.head()

,hotel_name,review_summary,score,review
0,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye..."
1,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye..."
2,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye..."
3,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye..."
4,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye..."


In [90]:
df.score.unique()

array([1, 2, 3, 4, 5])

In [91]:
#One Hot Encoding Scores Column

In [92]:
# For multi-class classifcation using the neural network model, 
# #I needed the target variable to be one hot encoded, as the output layer would have 5 nodes, 1 for each score.

In [93]:
enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(enc.fit_transform(df[['score']]).toarray())

df = df.join(enc_df)
df.head()

,hotel_name,review_summary,score,review,0,1,2,3,4
0,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
1,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
2,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
3,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
4,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0


In [94]:
df = df.rename(columns={0: "score_1", 1:'score_2',2:'score_3',3:'score_4',4:'score_5'})

In [95]:
df.head()

,hotel_name,review_summary,score,review,score_1,score_2,score_3,score_4,score_5
0,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
1,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
2,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
3,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0
4,"ITC Grand Chola, Chennai - a Luxury Collection...","ITC Grand Chola, Chennai - a Luxury Collection...",1,"Hi, My name is Vijay Jung Thapa - and I staye...",1.0,0.0,0.0,0.0,0.0


In [96]:
# Train and test split

In [97]:
x_train, x_test, y_train, y_test = train_test_split(df[['review','review_summary']], 
                                                    df[['score_1','score_2','score_3','score_4','score_5']], 
                                                    test_size=.2, 
                                                    random_state=42)

In [98]:
y_test.to_csv('/content/drive/MyDrive/NLP/6.y_test_ohe.csv',index=False)

In [99]:
#  Add Suffix to the Review Summary to Distinguish the Difference
# I wanted the words in the review summary to be separate from the words in the review,
#  as they may have a more or less significant impact. Also new data may not contain a review summary and 
#  the model
#  will not accurately reflect the review-only data.

In [100]:
x_train['review_summary_suffixed'] = x_train['review_summary'].apply(add_sum_suffix)
x_test['review_summary_suffixed'] = x_test['review_summary'].apply(add_sum_suffix)

In [101]:
x_train.head()

,review,review_summary,review_summary_suffixed
3994,Had a very comfortable stay at Taj Club house ...,Taj Club House,taj_sum club_sum house_sum
423,I am a hilton Honors silver member and have st...,Hilton Chennai,hilton_sum chennai_sum
2991,-Good hotel setup with average sized rooms. Ex...,The Park Chennai,the_sum park_sum chennai_sum
1221,I happen to be at Hilton for a dinner with a f...,Hilton Chennai,hilton_sum chennai_sum
506,The staff is very nice but big lack of trainin...,Hilton Chennai,hilton_sum chennai_sum


In [103]:
x_train['review_cleaned'] = x_train['review'].apply(text_cleanup)
x_test['review_cleaned'] = x_test['review'].apply(text_cleanup)

In [104]:
x_train

,review,review_summary,review_summary_suffixed,review_cleaned
3994,Had a very comfortable stay at Taj Club house ...,Taj Club House,taj_sum club_sum house_sum,had a very comfortable stay at taj club house ...
423,I am a hilton Honors silver member and have st...,Hilton Chennai,hilton_sum chennai_sum,i am a hilton honors silver member and have st...
2991,-Good hotel setup with average sized rooms. Ex...,The Park Chennai,the_sum park_sum chennai_sum,good hotel setup with average sized rooms exec...
1221,I happen to be at Hilton for a dinner with a f...,Hilton Chennai,hilton_sum chennai_sum,i happen to be at hilton for a dinner with a f...
506,The staff is very nice but big lack of trainin...,Hilton Chennai,hilton_sum chennai_sum,the staff is very nice but big lack of trainin...
...,...,...,...,...
1130,I was on a family visit to Chennai few weeks a...,Taj Club House,taj_sum club_sum house_sum,i was on a family visit to chennai few weeks a...
1294,We recently had a 2 night visit to The Park. T...,The Park Chennai,the_sum park_sum chennai_sum,we recently had a night visit to the park the ...
860,We had unfortunately stayed in this hotel for ...,"Lemon Tree Hotel, Chennai",lemon_sum tree_sum hotel_sum chennai_sum,we had unfortunately stayed in this hotel for ...
3507,"The hotel is located in the city, The hotel h...",The Park Chennai,the_sum park_sum chennai_sum,the hotel is located in the city the hotel has...


In [ ]:
#  Creating a Dictionary With Words That Appear in Reviews and an Index

In [105]:
# I created an index for each word in the training review dataset, so when passed through the embedding layer of the neural network there is an indexed number associated with each word.

In [106]:
flat_review = " ".join(x_train['review_cleaned'].values)
flat_review = flat_review.lower().split()

In [107]:
len(flat_review)

317498

In [108]:
flat_review_summary = " ".join(x_train['review_summary_suffixed'].values)
flat_review_summary = flat_review_summary.lower().split()

In [109]:
unique_list = []
for word in flat_review:
    if word not in unique_list:
        unique_list.append(word)
for word in flat_review_summary:
    if word not in unique_list:
        unique_list.append(word)

In [110]:
len(unique_list)

7242

In [111]:
word_index_dict = {}
for i in range(0,len(unique_list)):
    word_index_dict[unique_list[i]] = i

I added 4 other items to the dictionary that represented padding, the start of the review, unknown words, and unused words.

In [112]:
word_index_dict = {k:(v+4) for k,v in word_index_dict.items()}
word_index_dict['<PAD>'] = 0
word_index_dict['<START>'] = 1
word_index_dict['<UNK>'] = 2
word_index_dict['<UNUSED>'] = 3

Indexing Words in Reviews Using Dictionary

In [113]:
def index_review_words(text):
    review_word_list = []
    for word in text.lower().split():
        if word in word_index_dict.keys():
            review_word_list.append(word_index_dict[word])
        else:
            review_word_list.append(word_index_dict['<UNK>'])

    return review_word_list 

In [114]:
x_train['preprocessed_review'] = x_train['review_cleaned'].apply(index_review_words)
x_test['preprocessed_review'] = x_test['review_cleaned'].apply(index_review_words)

In [115]:
x_train['preprocessed_review_summary'] = x_train['review_summary_suffixed'].apply(index_review_words)
x_test['preprocessed_review_summary'] = x_test['review_summary_suffixed'].apply(index_review_words)

In [116]:
x_train



,review,review_summary,review_summary_suffixed,review_cleaned,preprocessed_review,preprocessed_review_summary
3994,Had a very comfortable stay at Taj Club house ...,Taj Club House,taj_sum club_sum house_sum,had a very comfortable stay at taj club house ...,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[7224, 7225, 7226]"
423,I am a hilton Honors silver member and have st...,Hilton Chennai,hilton_sum chennai_sum,i am a hilton honors silver member and have st...,"[42, 43, 5, 44, 45, 46, 47, 48, 49, 50, 14, 51...","[7227, 7228]"
2991,-Good hotel setup with average sized rooms. Ex...,The Park Chennai,the_sum park_sum chennai_sum,good hotel setup with average sized rooms exec...,"[109, 96, 110, 26, 111, 112, 25, 113, 114, 115...","[7229, 7230, 7228]"
1221,I happen to be at Hilton for a dinner with a f...,Hilton Chennai,hilton_sum chennai_sum,i happen to be at hilton for a dinner with a f...,"[42, 177, 19, 178, 9, 44, 33, 5, 179, 26, 5, 1...","[7227, 7228]"
506,The staff is very nice but big lack of trainin...,Hilton Chennai,hilton_sum chennai_sum,the staff is very nice but big lack of trainin...,"[15, 131, 103, 6, 200, 236, 243, 244, 30, 245,...","[7227, 7228]"
...,...,...,...,...,...,...
1130,I was on a family visit to Chennai few weeks a...,Taj Club House,taj_sum club_sum house_sum,i was on a family visit to chennai few weeks a...,"[42, 56, 63, 5, 68, 332, 19, 55, 457, 1414, 14...","[7224, 7225, 7226]"
1294,We recently had a 2 night visit to The Park. T...,The Park Chennai,the_sum park_sum chennai_sum,we recently had a night visit to the park the ...,"[69, 2213, 4, 5, 107, 332, 19, 15, 411, 15, 96...","[7229, 7230, 7228]"
860,We had unfortunately stayed in this hotel for ...,"Lemon Tree Hotel, Chennai",lemon_sum tree_sum hotel_sum chennai_sum,we had unfortunately stayed in this hotel for ...,"[69, 4, 182, 50, 14, 95, 96, 33, 5, 1321, 30, ...","[7231, 7232, 7233, 7228]"
3507,"The hotel is located in the city, The hotel h...",The Park Chennai,the_sum park_sum chennai_sum,the hotel is located in the city the hotel has...,"[15, 96, 103, 13, 14, 15, 16, 15, 96, 796, 152...","[7229, 7230, 7228]"


Combining Indexed Review Summary and Indexed Review Into a Single Column Called All Preprocessed Review

In [117]:
x_train['all_preprocessed_review'] = x_train['preprocessed_review'] + x_train['preprocessed_review_summary']
x_test['all_preprocessed_review'] = x_test['preprocessed_review'] + x_test['preprocessed_review_summary']

Modelling

 Add Padding to Reviews and Capping Reviews to Length 250 Words

In [118]:
x_train = sequence.pad_sequences(x_train['all_preprocessed_review'],value=word_index_dict['<PAD>'],padding='post',maxlen=250)
x_test = sequence.pad_sequences(x_test['all_preprocessed_review'],value=word_index_dict['<PAD>'],padding='post',maxlen=250)

Saving Neural Network Preprocessed x Data

In [119]:

np.save('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/6.x_test',x_test)

Creating and Training Neural Network Model

Here I created a neural network model with 1 embedding layer for the 17317 unique words
 (including the extra 4 from padding etc.), 1 dense layer with 16 nodes, and one output layer with 5 nodes for each score.

In [120]:
model = Sequential()
model.add(Embedding(17317,16))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(5,activation='sigmoid'))

In [121]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [122]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=.2,random_state=42)

In [125]:
fitModel = model.fit(x_train,y_train,epochs=100,batch_size=250,validation_data=(x_val,y_val),verbose=0)

Testing Model



After looking at accuracy scores for the validation data (0.5568) and the test data (0.5710), we can see the neural network model performs well.

In [126]:
model.evaluate(x_val,y_val)

20/20 [==============================] - 0s 2ms/step - loss: 0.2623 - accuracy: 0.9062


[0.2623198330402374, 0.90625]

In [127]:
model.evaluate(x_test,y_test)

25/25 [==============================] - 0s 2ms/step - loss: 0.2728 - accuracy: 0.9087


[0.2727559804916382, 0.9087499976158142]

In order to create a confusion matrix, I have to reverse one hot encode the scores column and adjust the index for 0-4 back to 1-5.

Reverse Encode Target Into One Column for Confusion Matrix

In [129]:
reverse_encode(y_test)

In [130]:
reverse_encode(y_val)

In [131]:
y_test_predicted = add_one_argmax_score(np.argmax(model.predict(x_test),axis=1))

Saving Neural Network Preprocessed y Data


In [132]:
y_test.to_csv('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/6.y_test_nn_df.csv',index=False)
np.save('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/6.y_test_predicted_array',y_test_predicted)

 Creating Confusion Matrix for Test Data

In [135]:
model.save('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/Models/Neural_Network.h5')

In [136]:
pickle.dump(word_index_dict,open('/content/drive/MyDrive/NLP/Hotel_Review_Classifier_/Models/word_index_dict.pkl','wb'))